W tym miejscu można znaleźć kod źródłowy i instrukcje dotyczące trenowania rekurencyjnej sieci neuronowej typu LSTM do generowania utworów muzycznych. Samo generowanie utworów zawiera skrypt `generuj.py`.

Przed rozpoczęciem pracy należy pamiętać o wyborze TPU jako urządzenia przyśpieszającego uczenie sieci: `Edit → Notebook settings → Hardware accelerator → TPU`.


In [1]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord
from keras.utils import np_utils

import os
import tensorflow as tf
# użyjemy trochę Kerasa (żeby było nieco czytelniej niż w czystym TF), ale nie używamy go w "podstawowej" wersji, tylko w wersji z tf.keras - dlaczego, o tym w dalszych komentarzach

Using TensorFlow backend.


Przed rozpoczęciem pracy należy pamiętać o ustawieniu odpowiednich ścieżek do plików wejściowych i wyjściowych:
*   `ścieżka_do_zbioru_uczącego` – pełna ścieżka do katalogu z plikami MIDI, na podstawie których będziemy uczyć naszą sieć, wraz ze wzorcem nazwy otwieranych plików (w tym wypadku – rozszerzenie `.mid`),
*   `plik_z_nutami` – plik, w którym zostanie zapisany zbiór dopuszczalnych nut i akordów (na podstawie zbioru uczącego),
*   `ścieżka_do_modelu` – plik, w którym zapiszemy wytrenowany model.

Korzystam tutaj z Dysku Google, a więc należy pamiętać przede wszystkim o załadowaniu plików na własny Dysk Google, jak również o tym, że należy używać ścieżek bezwzględnych, a więc z prefiksem `/content/gdrive/My Drive/`.


In [0]:
ścieżka_do_zbioru_uczącego = "/content/gdrive/My Drive/Zbiory dla Colaba/Utwory_MIDI/Klasyczna/Chopin/*.mid"
plik_z_nutami = "/content/gdrive/My Drive/Colab Notebooks/OMatKo 2019/nuty_chopin"
ścieżka_do_modelu = "/content/gdrive/My Drive/Colab Notebooks/OMatKo 2019/model_chopin.h5"

Ustawiamy też kilka parametrów, któr możemy zechcieć zmienić w ramach eksperymentów. Oczywiście można je ustawiać bezpośrednio w miejsach, w których będziemy z nich korzystać, ale jeśli są na górze, łatwiej je znaleźć.


*   `learning_rate` – współczynnik dotyczący pracy optymalizatora sieci neuronowej; zbyt mały spowoduje zbyt wolne uczenie sieci, zbyt duży – niestabilność uczenia, tzn. sieć może zbyt gwałtownie zmieniać wagi w końcowych iteracjach, kiedy dążymy już do pewnej stabilizacji na najlepszym możliwym poziomie;
*   `liczba_epok` – inaczej mówiąc liczba iteracji uczenia sieci,
*  `długość_sekwencji` – liczba nut i akordów, które będziemy rozpatrywać jako sekwencję dźwięków (każdy utwór podzielimy na kilka takich sekwencji).



In [0]:
learning_rate = 0.002
liczba_epok = 150
długość_sekwencji = 100

Poniższa komórka pobiera adres jednostki TPU. Używamy TPU z tego względu, iż uczenie sieci rekurencyjnych (różnego typu) trwa bardzo długo, a TPU pozwala to przyśpieszyć do akceptowalnego czasu.

Przykładowo, dla zbioru 100 plików MIDI i takiegj struktury sieci, jaką mamy w tym pliku, jedna epoka na CPU zajmuje ok. 80 minut, na GPU – 6-7 minut, na TPU natomiast 15-20 sekund. Jako że potrzebujemy około 150-180 epok, przyśpieszenie to jest naprawdę istotne.

In [4]:
try:
  nazwa_urządzenia = os.environ['COLAB_TPU_ADDR']
  ADRES_TPU = 'grpc://' + nazwa_urządzenia
  print('Znaleziono TPU pod adresem: {}'.format(ADRES_TPU))

except KeyError:
  print('Nie znaleziono TPU!')

Znaleziono TPU pod adresem: grpc://10.108.50.66:8470


Przed użyciem plików z Dysku Google, musimy zamontować nasz dysk. Po uruchomieniu tej komórki pojawi się link autoryzacyjny, gdyż trzeba będzie nadać Colabowi prawa do odczytu i zapisu plików na naszym dysku. Po nadaniu uprawnień uzyksamy kod autoryzacyjny, który trzeba wkleić w polu, które również pojawi się w tym miejscu, po czym należy kliknąć przycisk „Enter” na klawiaturze.

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
nuty = []

for plik in glob.glob(ścieżka_do_zbioru_uczącego): # otwieramy każdy plik ze zbioru uczącego
    midi = converter.parse(plik) # konwersja pliku MIDI na nuty i akordy, które możemy odczytać

    print('Przetwarzanie:', plik)

    nuty_do_przetworzenia = None

    # jeśli plik zawiera ścieżki dla kilku instrumentów
    try:
        partytura = instrument.partitionByInstrument(midi)
        nuty_do_przetworzenia = partytura.parts[0].recurse() 
    except:
        nuty_do_przetworzenia = midi.flat.notes

    for dźwięk in nuty_do_przetworzenia:
        if isinstance(dźwięk, note.Note): # jeśli to pojedyncza nuta - zapisz nazwę dźwięku
            nuty.append(str(dźwięk.pitch))
        elif isinstance(dźwięk, chord.Chord): # jeśli to akord - zapisz dźwięki rozdzielone kropkami (jako liczby, tu odsyłam do dokumentacji biblioteki music21)
            nuty.append('.'.join(str(n) for n in dźwięk.normalOrder))

# Utwory prztworzone na nuty zapisujemy do pliku - będzie to nam jeszcze potrzebne przy generowaniu utworów jako baza dla pierwszych dźwięków generowanych przez sieć
with open(plik_z_nutami, 'wb') as ścieżka:
    pickle.dump(nuty, ścieżka)

Przetwarzanie: /content/gdrive/My Drive/Zbiory dla Colaba/Utwory_MIDI/Klasyczna/Chopin/chpn_op35_4.mid
Przetwarzanie: /content/gdrive/My Drive/Zbiory dla Colaba/Utwory_MIDI/Klasyczna/Chopin/chpn_op23.mid
Przetwarzanie: /content/gdrive/My Drive/Zbiory dla Colaba/Utwory_MIDI/Klasyczna/Chopin/chpn_op7_1.mid
Przetwarzanie: /content/gdrive/My Drive/Zbiory dla Colaba/Utwory_MIDI/Klasyczna/Chopin/chpn-p3.mid
Przetwarzanie: /content/gdrive/My Drive/Zbiory dla Colaba/Utwory_MIDI/Klasyczna/Chopin/chpn_op25_e11.mid
Przetwarzanie: /content/gdrive/My Drive/Zbiory dla Colaba/Utwory_MIDI/Klasyczna/Chopin/chpn_op35_1.mid
Przetwarzanie: /content/gdrive/My Drive/Zbiory dla Colaba/Utwory_MIDI/Klasyczna/Chopin/chpn-p5.mid
Przetwarzanie: /content/gdrive/My Drive/Zbiory dla Colaba/Utwory_MIDI/Klasyczna/Chopin/chpn_op66.mid
Przetwarzanie: /content/gdrive/My Drive/Zbiory dla Colaba/Utwory_MIDI/Klasyczna/Chopin/chp_op31.mid
Przetwarzanie: /content/gdrive/My Drive/Zbiory dla Colaba/Utwory_MIDI/Klasyczna/Chopin/

In [0]:
długość_sekwencji = 100

# wszystkie nazwy dźwięków - potrzebujemy tylko listy "dopuszczalnych" wartości
nazwy_dźwięków = sorted(set(nuty))

liczba_dźwięków = len(nazwy_dźwięków) # liczba "dopuszczalnych" dźwięków i akordów w naszym "słowniku"

# mapowanie nazw dźwięków na liczby
nuty_int = dict((n, i) for i, n in enumerate(nazwy_dźwięków))

wejście_sieci = []
wyjście_sieci = []

for i in range(0, len(nuty) - długość_sekwencji):
    sekwencja_wejściowa = nuty[i:i + długość_sekwencji] # bierzemy kolejne x dźwięków z naszego zbioru (gdzie x jest ustaloną długością sekwencji)
    sekwencja_wyjściowa = nuty[i + długość_sekwencji] # i dźwięk, który po owej sekwencji nastąpi
    wejście_sieci.append([nuty_int[n] for n in sekwencja_wejściowa]) # do wejścia dodajemy liczbową reprezentację całej sekwencji (tu uwaga - dodajemy ją jako cały "rekord", więc kolejna sekwencja będzie już zupełnie oddzielona)
    wyjście_sieci.append(nuty_int[sekwencja_wyjściowa]) # do wyjścia dodajemy liczbową reprezentację wyjściowego dźwięku

liczba_wzorców = len(wejście_sieci) # ile "rekordów" mamy w naszej bazie - każdy rekord to sekwencja dźwięków + jeden dźwięk po niej rzeczywiście występujący

# trochę magii potrzebnej modelowi do poprawnego czytania danych
wejście_sieci = np.reshape(wejście_sieci, (liczba_wzorców, długość_sekwencji, 1))
# normalizacja
wejście_sieci = wejście_sieci / float(liczba_dźwięków)

# też trochę kerasowo-tensorflowowej magii potrzebnej ze względów formalno-technicznych, tylko tym razem do danych wyjściowych (tzn. oczekiwanych danych wyjściowych, prawidłowych klas - jak kto woli)
# dane wyjściowe muszą być podane w postaci wektora z tzw. kodowaniem gorącojedynkowym, tzn. takiego, gdzie mamy same zera oprócz pola odpowiadającego wartości przez ów wektor reprezentowanej - tam będzie stać jedynka
# (i to automatycznie zrobi funkcja to_categorical)
wyjście_sieci = np_utils.to_categorical(wyjście_sieci)

Powyżej można zauważyć pewną nieścisłość, wręcz małe „oszustwo” – które jednak nie tyle oszukuje nas, co raczej może wprowadzać w błąd sieć. Otóż zauważmy, co zrobiliśmy z naszymi utworami – połączyliśmy je w jedną wielką partyturę. A teraz – nie rozróżniając już tych utworów, dzielimy tę „superpartyturę” na sekwencje i czasami będziemy próbowali zmusić naszą sieć, żeby na podstawie ostatnich dźwięków poprzeniego utworu przewidziała pierwszy dźwięk kolejnego utworu. Czy to dobrze? Tak średnio, jest to dość sprzeczne z intuicją, choć może też pozwalać na ciekawe „zwroty akcji” w później generowanych partyturach. Niemniej – bezapelacyjnie lepiej byłoby te sekwencje podzielić tak, by sytuacje takie, jak wyżej wymieniona, się nie pojawiały. Jako że nie jest to trudne – pozostawiam to szanownemu Czytelnikowi jako proste ćwiczenie do sprawdzenia, czy poprawi to jakkolwiek wyniki. ;-)

(Należy jednak uważać na bardzo krótkie utwory, tzn. takie, dla których ustalona przez nas długość sekwencji będzie zawierała więcej nut niż ma sam utwór.)

In [8]:
# tworzony jest obiekt optymalizatora - można użyć też optymalizatora z tf.keras.optimiziers, ale ten jest nieco szybszy
opt = tf.train.RMSPropOptimizer(learning_rate)

# definicja modelu z użyciem Keras Functional API (przy Keras Sequential model nie moglibyśmy użyć TPU);
# poza tym należy poamiętać, że musimy używać warstw z tf.keras.layers, a nie z samego Kerasa (jak wyżej - inaczej nie zadziała)
warstwa_wejściowa=tf.keras.layers.Input(shape=(wejście_sieci.shape[1], wejście_sieci.shape[2]))
x=tf.keras.layers.LSTM(512, return_sequences=True)(warstwa_wejściowa)
x=tf.keras.layers.Dropout(0.3)(x)
x=tf.keras.layers.LSTM(512, return_sequences=True)(x) # każda warstwa LSTM prócz ostatniej musi mieć ustawiony parametr return_sequences=True
x=tf.keras.layers.Dropout(0.3)(x)
x=tf.keras.layers.LSTM(512)(x)
x=tf.keras.layers.Dense(256)(x)
x=tf.keras.layers.Dropout(0.3)(x)

# warstwa wyjściowa - potrzebujemy tylu neuronów, ile dźwięków jest dopuszczalnych w naszym modelu - 
# model będzie wybierać kolejne dźwięki na podstawie tego, z jakim dźwiękiem powiązany jest neuron zwracający największą wartość
warstwa_wyjściowa = tf.keras.layers.Dense(liczba_dźwięków, activation='softmax')(x)
model_f=tf.keras.models.Model(inputs=warstwa_wejściowa, outputs=warstwa_wyjściowa)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
model_f.compile(loss='categorical_crossentropy', optimizer=opt) # kompilacja modelu


# konwersja modelu biblioteki Keras na model TensorFlow przystosowany do pracy z TPU
tpu_model = tf.contrib.tpu.keras_to_tpu_model(model_f, strategy=tf.contrib.tpu.TPUDistributionStrategy(tf.contrib.cluster_resolver.TPUClusterResolver(ADRES_TPU)))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.108.50.66:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 10172020150407497454)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 4465260527371010534)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 11900663193035206448)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:

In [0]:
# Tworzymy callback do checkpointingu - za każdym razem, kiedy powstanie model o mniejszej wartości funkcji straty - a więc lepiej przystosowany - będzie on zapisywany
# daje nam to też możliwość zakończenia procesu uczenia przed upływem wszystkich iteracji, jeśli zauważymy, że model już nie ma zbyt dużych perspektyw na faktyczną poprawę jakości
callback_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    ścieżka_do_modelu,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

In [11]:
# trenowanie sieci
tpu_model.fit(wejście_sieci, wyjście_sieci, epochs=liczba_epok, batch_size=1024, callbacks=[callback_checkpoint])

Epoch 1/150
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(128,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(128, 100, 1), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(128, 317), dtype=tf.float32, name='dense_1_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 9.49458932876587 secs
INFO:tensorflow:Setting weights on TPU model.
61440/63329 [============================>.] - ETA: 0s - loss: 5.7510INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(108,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(108, 100, 1), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(108, 317), dtype=tf.float32, name='dense_1_target_30')]
INFO:tensor